In [1]:
# using the method according to jinjin's paper
import pandas as pd
import numpy as np
from scipy.stats import lognorm

In [2]:
df = pd.read_csv('C:/Users/mchen48/Box/01 Research/PFASs/PFASs_in_Carpet/00 PFAS_US_carpet/PFAS_flows_stocks_data_processing/Basic_data.csv')

In [3]:
# rao input parameters such as the concentration of different PFAS at different scenarios
# the unit of all V parameters are million m2
P_import = 79     # change according to different type of PFAS, ug/m2
P_production  = 10500  # change according to different type of PFAS, ug/m2
P_export = P_production   # change according to different type of PFAS, ug/m2
R_to_carpet = 0.36   # the ratio of collected carpet recycled to carpet
P_yearlyloss_air = 729.813  # change according to different type of PFAS, ug/m2/year. As the specific situation, see the note of PFAS calculation
P_yearlyloss_dust = 0.03942     # change according to different type of PFAS, ug/m2/year
Prod_emi_R = 0.15 # High percentage loss of PFAS during production, related to F9, the high is 0.15, the low is 0.05
xt = 20.6 # ug/L, the average PFAS concentration in landfill leachate from discarded carpets (Table 1 in Jinjin's paper)
SA = 140 # 140 +- 98 tonnes/ha, average mass of waste per surface area for U.S. landfills
LG = 3.26 # m3/ha/day based on the annual precipitation, calculate national leachate generation rate 
R_pro_waste = 0.018 # the material waste during carpet production

In [4]:
# creat a lognormal distribution
age_iniStock = 4.3  # the initial age of nylon carpet stock (in 1990) is 4.3 years old
shape = 0.2957397   # standard deviation
scale = np.exp(2.1080312) # mean
loc = 0
dist = lognorm(shape, loc, scale)

In [5]:
# calculate In-Use Stock of PFAS in nylon carpet
Af = 159 # the average single family house size before 1990 is 159 m2
P_cf = 0.51
S0_volumn = (Af/3.35) * P_cf * df.loc[0,'us_pop'] * df.loc[0,'Percent_nylon'] # Calculate the initial carpet stock volumn in use (initial average family size 3.35)
S0_PFAS  = (S0_volumn * P_production * 0.85)/1000  # Calculate the initial PFAS stock in use

In [6]:
# creat the column of F1_NP_High
df['F1'] = 0 ## the unit of F1_SP_High is kg

In [7]:
# calculate F1: PFAS flow in imported nylon carpet
for i in range(len(df)):
    df.loc[i,'F1'] = (df.loc[i,'V_import'] * P_import * df.loc[i,'Percent_nylon'] - 
                              df.loc[i,'V_export'] * P_export * df.loc[i,'Percent_nylon'])/1000

In [8]:
# creat the column of F_production
df['F_production'] = 0 ## the unit of F_production is kg

In [9]:
# calculate F_production:PFAS flow used in the domestic carpet production in nylon carpet
for i in range(len(df)):
    df.loc[i,'F_production'] = (df.loc[i,'V_production'] * P_production * df.loc[i,'Percent_nylon']*df.loc[i,'PhaseOut'])/1000

In [10]:
df['F3'] = 0 

In [11]:
# calculate F3:PFAS flow in material loss in production
for i in range(len(df)):
    df.loc[i,'F3'] = df.loc[i,'F_production'] * 0.12 * 0.15

In [12]:
df_yearly_EoL = pd.DataFrame() #creat an empty dataframe to store every year's EoL carpet according to lognormal distribution

In [13]:
for year in range(1990,2061):
    df_yearly_EoL[year] = 0

In [14]:
for i in range(0,71):
    df_yearly_EoL.loc[i,:] = 0

In [15]:
for i in range(0,71):
    for year in range(1990,2061):
        if year >= df.loc[i,'year']:
            df_yearly_EoL.loc[i,year] = (dist.cdf(year - df.loc[i,'year'] + 1) - dist.cdf(year - df.loc[i,'year'])) * df.loc[i,'V_sales_nylon']
        else:
            df_yearly_EoL.loc[i,year] = df_yearly_EoL.loc[i,year]

In [16]:
df_yearly_EoL

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060
0,2.660423e-10,4.484103e-04,1.672750e-01,3.664329e+00,2.015205e+01,50.435212,78.026173,88.668697,82.040428,65.916845,...,1.050359e-09,7.121885e-10,4.844019e-10,3.306300e-10,2.262785e-10,1.553961e-10,1.070777e-10,7.395670e-11,5.121864e-11,3.561523e-11
1,0.000000e+00,2.692118e-10,4.537522e-04,1.692678e-01,3.707983e+00,20.392121,51.036056,78.955714,89.725024,83.017790,...,1.572884e-09,1.062872e-09,7.206730e-10,4.901727e-10,3.345688e-10,2.289742e-10,1.572473e-10,1.083533e-10,7.483776e-11,5.182882e-11
2,0.000000e+00,0.000000e+00,2.724138e-10,4.591492e-04,1.712811e-01,3.752086,20.634666,51.643082,79.894819,90.792220,...,2.363185e-09,1.591592e-09,1.075514e-09,7.292447e-10,4.960028e-10,3.385482e-10,2.316976e-10,1.591177e-10,1.096421e-10,7.572789e-11
3,0.000000e+00,0.000000e+00,0.000000e+00,2.754294e-10,4.642320e-04,0.173177,3.793622,20.863094,52.214776,80.779262,...,3.560058e-09,2.389346e-09,1.609211e-09,1.087420e-09,7.373175e-10,5.014936e-10,3.422960e-10,2.342625e-10,1.608791e-10,1.108558e-10
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.781956e-10,0.000469,0.174916,3.831722,21.072626,52.739179,...,5.376280e-09,3.595812e-09,2.413343e-09,1.625373e-09,1.098341e-09,7.447225e-10,5.065302e-10,3.457337e-10,2.366153e-10,1.624948e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.780270e-10,4.686102e-04,1.748104e-01,3.829400e+00,2.105985e+01
67,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.791231e-10,4.704577e-04,1.754996e-01,3.844497e+00
68,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.802290e-10,4.723217e-04,1.761950e-01
69,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.813459e-10,4.742041e-04


In [17]:
# creat a column to store the EoL of initial stock, lognormal distribution, initial age is 4.3 years
# mean and std are the same as yearly EoL of sales volumn
df['V_iniStock_EoL'] = 0

In [18]:
for i in range(0,71):
    df.loc[i,'V_iniStock_EoL'] = ((dist.cdf(age_iniStock+i+1) - dist.cdf(age_iniStock+i))/
                                  (1-dist.cdf(age_iniStock)))*S0_volumn       

In [19]:
# generate a column to store the yearly EoL from every year's new sales to the market
df['V_new_EoL'] = 0

In [20]:
# calculate the yearly end of life carpet volumn
b = 0
for year in range(1990, 2061):
    a = 0
    for i in range(0,71):
        a = a + df_yearly_EoL.loc[i,year]
    df.loc[b,'V_new_EoL'] = a
    b = b + 1

### Calculate v4_phaseout, which means under the phaseout scenario

In [21]:
# generate a column to store the yearly EoL from every year's new sales to the market in phase out scenario
df['V_new_EoL_PhaseOut'] = 0

In [22]:
b = 0
for year in range(1990, 2061):
    a = 0
    for i in range(0,71):
        a = a + df_yearly_EoL.loc[i,year]*df.loc[i,'PhaseOut']
    df.loc[b,'V_new_EoL_PhaseOut'] = a
    b = b + 1

In [23]:
# generate a column to store the total EoL volumn from stock carpet and every year's new carpet
df['V4_PhaseOut'] = 0

In [24]:
for i in range(0,71):
    df.loc[i,'V4_PhaseOut'] = df.loc[i,'V_iniStock_EoL'] + df.loc[i,'V_new_EoL_PhaseOut']

In [25]:
df['V_delta_S_PhaseOut'] = 0 # add a column to stock yearly nylon carpet volumn stock change under phaseout scenario

In [26]:
# calculate the yearly change of nylon carpet in use stock under phaseout scenario
for i in range(0,71):
     df.loc[i,'V_delta_S_PhaseOut'] = df.loc[i,'V_sales_nylon']*df.loc[i,'PhaseOut'] - df.loc[i,'V4_PhaseOut']

In [27]:
df['V_stock_PhaseOut'] = 0 # add a column to stock yearly total nylon carpet volumn stock 

In [28]:
# calculate the total carpet in stock every year
df.loc[0,'V_stock_PhaseOut'] = S0_volumn + df.loc[0,'V_delta_S_PhaseOut']
for i in range(1,71):
     df.loc[i,'V_stock_PhaseOut'] = df.loc[i-1,'V_stock_PhaseOut'] + df.loc[i, 'V_delta_S_PhaseOut']

#### calculate normal v4

In [29]:
# generate a column to store the total EoL volumn from stock carpet and every year's new carpet
df['V4'] = 0

In [30]:
for i in range(0,71):
    df.loc[i,'V4'] = df.loc[i,'V_iniStock_EoL'] + df.loc[i,'V_new_EoL']

In [31]:
df['V_delta_S'] = 0 # add a column to stock yearly nylon carpet volumn stock change

In [32]:
# calculate the yearly change of nylon carpet in use stock
for i in range(0,71):
     df.loc[i,'V_delta_S'] = df.loc[i,'V_sales_nylon'] - df.loc[i,'V4']

In [33]:
df['V_stock'] = 0 # add a column to stock yearly total nylon carpet volumn stock 

In [34]:
# calculate the total carpet in stock every year
df.loc[0,'V_stock'] = S0_volumn + df.loc[0,'V_delta_S']
for i in range(1,71):
     df.loc[i,'V_stock'] = df.loc[i-1,'V_stock'] + df.loc[i, 'V_delta_S']

In [35]:
df['F_to_air'] = 0 # add a column to stock yearly in use PFAS lost to air

In [36]:
df['F_to_dust'] = 0 # add a column to stock yearly in use PFAS lost to dust

In [37]:
df['F6'] = 0 # add a column to stock yearly PFAS emission from in use carpet

In [38]:
# calculate the PFAS emissions from in-use stock
for i in range(0,71):
    df.loc[i,'F_to_air'] = P_yearlyloss_air * df.loc[i,'V_stock_PhaseOut']/1000
    df.loc[i,'F_to_dust'] = P_yearlyloss_dust * df.loc[i,'V_stock_PhaseOut']/1000
    df.loc[i,'F6'] = (df.loc[i,'F_to_air'] + df.loc[i,'F_to_dust'])*2

### In_use stock calculation

In [39]:
df['F_delta_S'] = 0 # add a column to stock yearly total PFAS change in stock

In [40]:
df['F_stock'] = 0 # add a column to stock yearly total PFAS in stock

In [41]:
df['F2'] = 0
df['F4'] = 0
df['F5'] = 0

In [42]:
#method 2: using the method in excel template, according to the potion of in use carpet to calculate
df.loc[0,'F4'] = (df.loc[0,'V4']/S0_volumn)*S0_PFAS
df.loc[0,'F2'] = df.loc[0,'F1'] + df.loc[0,'F_production'] + df.loc[0,'F5']- df.loc[0,'F3']
df.loc[0,'F_delta_S'] = df.loc[0,'F2'] - df.loc[0,'F4'] - df.loc[0,'F6']
df.loc[0,'F_stock'] = df.loc[0,'F_delta_S'] + S0_PFAS
for i in range(1,71):
    if df.loc[i-1, 'F_stock'] > 0:
        df.loc[i,'F4'] = (df.loc[0,'V4']/df.loc[i-1,'V_stock'])*df.loc[i-1,'F_stock']
        df.loc[i,'F5'] = df.loc[i-1,'F4'] * df.loc[i,'RR_total'] * R_to_carpet
        df.loc[i,'F2'] = df.loc[i,'F1'] + df.loc[i,'F_production'] + df.loc[i,'F5']- df.loc[i,'F3']
        df.loc[i,'F_delta_S'] = df.loc[i,'F2'] - df.loc[i,'F4'] - df.loc[i,'F6']
        df.loc[i,'F_stock'] = df.loc[i,'F_delta_S'] + df.loc[i-1,'F_stock']
    elif df.loc[i-1, 'F_stock'] < 0:
        df.loc[i-1,'F_stock'] = df.loc[i-2,'F_stock']
        df.loc[i,'F4'] = (df.loc[0,'V4']/df.loc[i-1,'V_stock'])*df.loc[i-1,'F_stock']
        df.loc[i,'F5'] = df.loc[i-1,'F4'] * df.loc[i,'RR_total'] * R_to_carpet
        df.loc[i,'F2'] = df.loc[i,'F1'] + df.loc[i,'F_production'] + df.loc[i,'F5']- df.loc[i,'F3']
        df.loc[i,'F_delta_S'] = df.loc[i,'F2'] - df.loc[i,'F4'] - df.loc[i,'F6']
        df.loc[i,'F_stock'] = df.loc[i,'F_delta_S'] + df.loc[i-1,'F_stock']

In [43]:
df['F8'] = 0

In [44]:
# calculate the PFAS flow in discarded carpet to incineration
for i in range(0,71):
    df.loc[i,'F8'] = df.loc[i,'F4'] * df.loc[i,'IncR']

In [45]:
df['F9'] = 0

In [46]:
# calculate the PFAS emission during carpet production
for i in range(0,71):
    df.loc[i,'F9'] = df.loc[i,'F_production'] * Prod_emi_R

### Calculation of PFASs accumulation in landfills

In [47]:
df['F_EoL_total'] = 0

In [48]:
# calculate the yearly total EoL, including F4 and F3
for i in range(0,71):
    df.loc[i,'F_EoL_total'] = df.loc[i,'F3'] + df.loc[i,'F4']

In [49]:
df['landfill_ratio'] = 0 # the ratio is calculated using 1 - recycle ratio - incineration ratio

In [50]:
for i in range(0,71):
    df.loc[i,'landfill_ratio'] = 1 - df.loc[i,'RR_total'] - df.loc[i,'IncR']

In [51]:
df['F_landfill_yearly'] = 0 # store every year's landfill PFAS

In [52]:
# calculate the yearly landfill PFAS
for i in range(0,71):
    df.loc[i,'F_landfill_yearly'] = df.loc[i,'F_EoL_total'] * df.loc[i,'landfill_ratio']

In [53]:
df['F_accu_landfill'] = 0 # store accumulated landfill PFAS, in unit of tonnes

In [54]:
# calculate the accumulated PFAS to landfill for every year
for i in range(0,71):
    a = 0
    for j in range(0,i+1):
        a = a + df.loc[j,'F_landfill_yearly']
    df.loc[i,'F_accu_landfill'] = a

### Caclulation of PFASs in landfill leachate

In [55]:
df_yearly_EoL_PhaseOut = df_yearly_EoL

In [56]:
for i in range(0,71):
    for year in range(1990,2061):
        df_yearly_EoL_PhaseOut.loc[i,year] = df_yearly_EoL_PhaseOut.loc[i,year] * df.loc[i,'PhaseOut']

In [57]:
df['V_new_EoL_PhaseOut'] = 0

In [58]:
b = 0
for year in range(1990, 2061):
    a = 0
    for i in range(0,71):
        a = a + df_yearly_EoL_PhaseOut.loc[i,year]
    df.loc[b,'V_new_EoL_PhaseOut'] = a
    b = b + 1

In [59]:
df['V4_PhaseOut'] = 0

In [60]:
for i in range(0,71):
    df.loc[i,'V4_PhaseOut'] = df.loc[i,'V_iniStock_EoL'] + df.loc[i,'V_new_EoL_PhaseOut']

In [61]:
df['V_annual_landfill'] = 0 # million m2, creat a column to store the volumn of end of life carpet, including discarded carpet and waste from production.

In [62]:
# calculate the annual volumn of carpet to landfill, including in use EoL and production waste.
for i in range(0,71):
    df.loc[i,'V_annual_landfill'] = df.loc[i,'V4_PhaseOut'] * df.loc[i,'landfill_ratio'] + df.loc[i,'V_production'] * df.loc[i,'Percent_nylon'] * R_pro_waste * df.loc[i,'PhaseOut']

In [63]:
df['V_accum_landfill'] = 0 # million m2, creat a column to store the volumn of accumulated carpet in landfills

In [64]:
# calculate the accumulated volumn of carpet to landfill for every year
for i in range(0,71):
    a = 0
    for j in range(0,i+1):
        a = a + df.loc[j,'V_annual_landfill']
    df.loc[i,'V_accum_landfill'] = a

In [65]:
df['Mass_accum_landfill'] = 0 # tonnes, the mass of carpet accumulated in landfill

In [66]:
for i in range(0,71):
    df.loc[i,'Mass_accum_landfill'] = df.loc[i,'V_accum_landfill'] * 23.9 * 1000

In [67]:
df['PFAS_landfill_leachate'] = 0 # Kg, the mass of PFAS leachated from landfilled carpet

In [68]:
for i in range(0,71):
    df.loc[i,'PFAS_landfill_leachate'] = df.loc[i,'Mass_accum_landfill'] * xt * LG/140 * 365/1000000 * 0.35

In [69]:
df.to_csv('NP_HIGH_PhaseOut_v2.csv',index = False)